## Run with "myenv"

In [ ]:
import bpy

# Ensure the VRM add-on is enabled
bpy.ops.preferences.addon_enable(module='cats-blender-plugin')

# Load the VRM model
vrm_filepath = "Datasets/3D_VRMModel/model.vrm"
bpy.ops.import_scene.vrm(filepath=vrm_filepath)

# Export the model to OBJ format
obj_filepath = "Datasets/3D_OBJModel/model.obj"
bpy.ops.export_scene.obj(filepath=obj_filepath)

# Load the exported OBJ model
bpy.ops.import_scene.obj(filepath=obj_filepath)

# Set up the scene
scene = bpy.context.scene

# Set up the camera for front view
cam = bpy.data.objects['Camera']
cam.location = (0, -5, 1.5)  # Adjust the Y and Z coordinates as necessary
cam.rotation_euler = (1.5708, 0, 0)  # 90 degrees in radians for front view

# Set up lighting
light_data = bpy.data.lights.new(name="SunLight", type='SUN')
light_object = bpy.data.objects.new(name="SunLight", object_data=light_data)
scene.collection.objects.link(light_object)
light_object.location = (5, -5, 5)
light_object.rotation_euler = (1.5708, 0, 0.7854)  # Adjust as needed

# Create a Toon Shader
for obj in scene.objects:
    if obj.type == 'MESH':
        mat = bpy.data.materials.new(name="ToonShader")
        mat.use_nodes = True
        nodes = mat.node_tree.nodes
        links = mat.node_tree.links

        # Clear default nodes
        for node in nodes:
            nodes.remove(node)

        # Add Toon Shader nodes
        output_node = nodes.new(type='ShaderNodeOutputMaterial')
        diffuse_node = nodes.new(type='ShaderNodeBsdfDiffuse')
        toon_shader = nodes.new(type='ShaderNodeShaderToRGB')
        color_ramp = nodes.new(type='ShaderNodeValToRGB')

        # Position nodes
        output_node.location = (300, 0)
        toon_shader.location = (0, 0)
        diffuse_node.location = (-300, 0)
        color_ramp.location = (150, 0)

        # Link nodes
        links.new(diffuse_node.outputs[0], toon_shader.inputs[0])
        links.new(toon_shader.outputs[0], color_ramp.inputs[0])
        links.new(color_ramp.outputs[0], output_node.inputs[0])

        # Adjust color ramp for toon effect
        color_ramp.color_ramp.elements[0].position = 0.4
        color_ramp.color_ramp.elements[1].position = 0.6

        # Assign material to object
        obj.data.materials.append(mat)

# Render settings
scene.render.image_settings.file_format = 'PNG'
scene.render.filepath = '/path/to/output/image.png'
scene.render.resolution_x = 1920
scene.render.resolution_y = 1080

# Render the image
bpy.ops.render.render(write_still=True)
